## create a dataframe used to create all visualizations for that recommendation



Function used to fill in the dataframe
If the concept name and record name exist in a row of the dataDF return a one. If they do not exist, check to see if the concept is contained in the dialect using the dialectDF (use filepath to determine dialect). If yes, return zero. if not in dialect return negative one

Import as needed

In [95]:
import pandas as pd
import pixiedust
import holoviews as hv
import numpy as np

import os
from os import walk
import fnmatch
from ipywidgets import *
import ipywidgets as widgets

hv.__version__

1.8.4

### Select evaluation of a metadata collection and prepare it for analysis
1. Query the data directory for evaluated data and use the results to populate a list 
2. Function that reads the selected csv into a dataframe

In [96]:
# 0.


# 1.
DataFiles=[]
for path, subdirectories, filenames in os.walk('../data/'):
    for filename in filenames:
        if fnmatch.fnmatch(filename, '*Evaluated.csv'):
            DataFiles.append(os.path.join(path,filename).split("../data/", 1)[-1])
# 2.    
def DataChoices(DataFile):
    global EvaluatedMetadataDF
    global dataFile
    dataFile=DataFile
    EvaluatedMetadataDF= pd.read_csv(os.path.join('../data', DataFile))
    return EvaluatedMetadataDF
# 3.


read in the EvaluatedMetadata of the collection you want to analyze

In [97]:
interactive(DataChoices,DataFile=DataFiles)


In [98]:
dataFile

'LTERthroughTime/LTER_2007_EML_Evaluated.csv'

In [78]:
EvaluatedMetadataDF

,Concept,Content,Record,XPath
0,Abstract,"Nanomolar concentrations of PO4, NO3, NO2 (sur...",dataset_3470.xml,/gmi:MI_Metadata/gmd:identificationInfo/gmd:MD...
1,Acknowledgement,Funding provided by NSF Ocean Sciences (NSF OC...,dataset_3470.xml,/gmi:MI_Metadata/gmd:identificationInfo/gmd:MD...
2,Acknowledgement,Funding provided by NSF Ocean Sciences (NSF OC...,dataset_3470.xml,/gmi:MI_Metadata/gmd:identificationInfo/gmd:MD...
3,Address,WHOI MS#36,dataset_3470.xml,/gmi:MI_Metadata/gmd:contact/gmd:CI_Responsibl...
4,Address,"Department of Ocean, Earth, and Atmospheric Sc...",dataset_3470.xml,/gmi:MI_Metadata/gmd:identificationInfo/gmd:MD...
5,Address,WHOI MS#36,dataset_3470.xml,/gmi:MI_Metadata/gmd:contentInfo/gmd:MD_Featur...
6,Address,WHOI MS#36,dataset_3470.xml,/gmi:MI_Metadata/gmd:distributionInfo/gmd:MD_D...
7,Address,WHOI MS#36,dataset_3470.xml,/gmi:MI_Metadata/gmd:metadataMaintenance/gmd:M...
8,AssociatedDIFs,U.S. GEOTRACES,dataset_3470.xml,/gmi:MI_Metadata/gmd:identificationInfo/gmd:MD...
9,AssociatedDIFs,U.S. GEOTRACES NAT,dataset_3470.xml,/gmi:MI_Metadata/gmd:identificationInfo/gmd:MD...


Create a table with each record as a row of concept occurance counts. Each concept that occurs in the collection is a row.

In [80]:
FiledirectoryRAD=os.path.join('../data/',Organization)
FilenameRAD='/'+Collection+'_'+Dialect+'_RAD.csv'
FilePathRAD=FiledirectoryRAD+FilenameRAD
FilePathRAD
group_name = EvaluatedMetadataDF.groupby(['Record', 'Concept'], as_index=False)
occuranceMatrix=group_name.size().unstack().reset_index()
occuranceMatrix=occuranceMatrix.fillna(0)
occuranceMatrix.to_csv(FilePath, mode = 'w', index=False)

NameError: name 'Organization' is not defined

Sum up the collection concept occurance 
Count up how many records contain the concept.
Combine the two tables and add a collection, collection occurance, and average record occurance columns

In [94]:
FiledirectoryOccurance=os.path.join('../data/',Organization)
FilenameOccurance='/'+Collection+'_'+Dialect+'_Occurance.csv'
FilePathOccurance=FiledirectoryOccurance+FilenameOccurance
occuranceSum=occuranceMatrix.sum()
occuranceCount=occuranceMatrix[occuranceMatrix!=0].count()
CollectionName=collectionDialect=dataFile.partition("/")[2].partition("_Evaluated.csv")[0]
result = pd.concat([occuranceSum, occuranceCount], axis=1).reset_index()
result.insert(1, 'Collection', CollectionName)
result.insert(4, 'CollectionOccurance%', CollectionName)
result.insert(4, 'AverageOccurancePerRecord', CollectionName)
result.columns = ['Concept', 'Collection', 'ConceptCount', 'RecordCount', 'AverageOccurancePerRecord', 'CollectionOccurance%' ]
NumberOfRecords = result.at[0, 'ConceptCount'].count('.xml')
result['CollectionOccurance%'] = result['RecordCount']/NumberOfRecords
result['CollectionOccurance%'] = pd.Series(["{0:.2f}%".format(val * 100) for val in result['CollectionOccurance%']], index = result.index)

result.at[0, 'ConceptCount'] = NumberOfRecords
result.at[0, 'Concept'] = 'Number of Records'
result['AverageOccurancePerRecord'] = result['RecordCount']/NumberOfRecords
result.to_csv(FilePathOccurance, mode = 'w', index=False)
result

NameError: name 'Organization' is not defined